In [1]:
# Re-import after code state reset
from faker import Faker
import pandas as pd
import sqlite3
import uuid
import random
from datetime import datetime

# Initialize Faker
fake = Faker()

# Generate synthetic customer data
def generate_customers(n=70):
    data = []
    for _ in range(n):
        customer = {
            "id": str(uuid.uuid4()),
            "full_name": fake.name(),
            "email": fake.email(),
            "phone_number": fake.phone_number(),
            "address": fake.address().replace("\n", ", "),
            "account_number": fake.unique.bban(),
            "account_type": random.choice(["Savings", "Cheque", "Business"]),
            "balance": round(random.uniform(100.0, 50000.0), 2),
            "loan_eligible": random.choice([True, False]),
            "loan_amount": round(random.uniform(0.0, 200000.0), 2),
            "last_login": fake.date_time_between(start_date="-1y", end_date="now").strftime("%Y-%m-%d %H:%M:%S"),
        }
        data.append(customer)
    return pd.DataFrame(data)

# Generate the data
df_customers = generate_customers()

# Save to SQLite database in the Jupyter environment
conn = sqlite3.connect("local_customers.db")
df_customers.to_sql("customers", conn, index=False, if_exists="replace")
conn.commit()
conn.close()

# import ace_tools as tools; tools.display_dataframe_to_user(name="Local Synthetic Customer Data", dataframe=df_customers)


In [2]:
# Step 1: Set up database connection for reuse
import sqlite3

# Connect to the local SQLite DB
db_path = "local_customers.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


# Function to update customer details
def update_customer_detail(identifier, field, new_value):
    query = f"""
    UPDATE customers SET {field} = ? WHERE id = ? OR account_number = ?
    """
    cursor.execute(query, (new_value, identifier, identifier))
    conn.commit()
    return cursor.rowcount > 0

# Confirm that everything is ready
"Database connection and utility functions are set up. Ready for GPT chatbot integration."


'Database connection and utility functions are set up. Ready for GPT chatbot integration.'

In [12]:
import openai
import os
from openai import OpenAI
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Sql functions

In [25]:
def get_balance(customer_id: str) -> dict:
    """
    Query the local_customers.db for this customer_id 
    and return a dict with name & balance (or an error).
    """
    conn = sqlite3.connect("local_customers.db")
    cur = conn.cursor()
    cur.execute(
        "SELECT full_name, balance FROM customers WHERE id = ?", 
        (customer_id,)
    )
    row = cur.fetchone()
    conn.close()

    if row:
        return {"found": True, "full_name": row[0], "balance": row[1]}
    else:
        return {"found": False}

In [103]:
df_customers.head()

,id,full_name,email,phone_number,address,account_number,account_type,balance,loan_eligible,loan_amount,last_login
0,bd9c5311-c8f8-441d-b205-e68fb7aa7293,Hayley Taylor,lindsayevans@example.com,240.730.7657,"66873 Johns Station Apt. 951, New Cheryl, PW 8...",LSEJ75490545720505,Cheque,28221.70,True,62271.66,2024-09-01 07:44:23
1,4f8667b3-f381-44c4-8f3c-2a0718248284,Veronica Wilson,peggyhenderson@example.com,(933)323-1013x26794,"492 Nunez Field, East Julia, MN 48082",HPJO70182321326256,Savings,1342.67,True,117454.65,2024-05-28 02:03:26
2,a217997d-21c3-4716-ac19-5d5c9950c260,Kim Taylor,aaronclark@example.org,2653847718,"2615 Stewart View, Davilaside, MN 11472",PSRT26578460342140,Business,14389.60,False,117932.25,2025-03-31 13:01:01
3,b2ad5dfb-e86e-460d-9f98-3351c040e02f,Mr. Robert Nicholson,rebecca59@example.net,+1-988-303-9632x1080,"Unit 9188 Box 7339, DPO AE 49110",XUWR46473567505370,Cheque,21475.10,True,175559.46,2024-10-03 14:58:24
4,9e5d54aa-168e-4347-8514-9ec8d1d5a470,Alicia Jones,anthonymurphy@example.com,798-968-5221x242,"9544 Brown Island Apt. 484, Lake Calvin, ND 85959",JOXB12585286300382,Business,13290.37,True,168622.08,2025-03-09 22:31:15


In [108]:
import sqlite3

def get_loan_eligibility(customer_id: str) -> dict:
    """
    Query the local_customers.db for this customer_id 
    and return a dict with eligibility, loan amount, and name.
    """
    conn = sqlite3.connect("local_customers.db")
    cur = conn.cursor()
    cur.execute(
        "SELECT full_name, loan_eligible, loan_amount FROM customers WHERE id = ?", 
        (customer_id,)
    )
    row = cur.fetchone()
    conn.close()

    if row:
        return {
            "found": True,
            "full_name": row[0],
            "eligible": row[1],
            "max_loan_amount": row[2]
        }
    else:
        return {"found": False}


# Function calling

In [109]:
function_descriptions = [{
        "type": "function",
        "name": "get_balance",
        "description": "Retrieve a customer's name and balance from the SQLite database",
        "parameters": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The ID of the customer for retrieving information from database"
                },
                
            },
            "required": ["customer_id"],
            "additionalProperties": False
        },
    },
    {
    "type": "function",
    "name": "get_loan_eligibility",
    "description": "Check if a user qualifies for a loan and how much they qualify for",
    "parameters": {
        "type": "object",
        "properties": {
            "customer_id": {
                "type": "string",
                "description": "The unique ID of the customer"
            },
            "requested_amount": {
                "type": "number",
                "description": "The loan amount the customer wants to borrow"
            }
        },
        "required": ["customer_id", "requested_amount"]
    }
}]

# Prompts

In [111]:
combined_prompt = """
You are a friendly and helpful banking assistant.

You can help users with two tasks:
1. Checking their account balance.
2. Applying for a loan.

If the user wants to check their balance:
- Ask for their customer ID.
- Then call the balance function to retrieve their name and balance.
- Respond directly to the user with their name and balance.

If the user wants to apply for a loan:
- Ask how much they want to borrow.
- Then ask for their customer ID.
- Call the loan eligibility function to check if they qualify and how much they qualify for.
- If they qualify for the requested amount, respond like: "Hi [Name], you qualify for a loan of R[amount]."
- If they qualify for less, respond like: "Hi [Name], unfortunately you don’t qualify for R[amount], but you do qualify for R[max_loan_amount]."
- If they don’t qualify, respond politely with: "Hi [Name], unfortunately you are not eligible for a loan."

Always speak directly to the user and use a polite, helpful tone.
"""


# Routing agent

In [117]:
pip install --upgrade sqlalchemy


     |████████████████████████████████| 2.1 MB 10.1 MB/s eta 0:00:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.47
    Uninstalling SQLAlchemy-1.4.47:
      Successfully uninstalled SQLAlchemy-1.4.47
Note: you may need to restart the kernel to use updated packages.


In [118]:
from crewai import Agent, Crew


ImportError: cannot import name 'Result' from 'sqlalchemy' (/Users/mahlatsemotsamai/opt/anaconda3/lib/python3.8/site-packages/sqlalchemy/__init__.py)

In [ ]:

# Define agents
router = Agent(name="Router", role="Routing Agent", goal="Decide which agent to use", tools=[], ...)
balance_agent = Agent(name="Balance Agent", role="Balance Checker", goal="Return user balance", tools=[balance_tool], ...)
loan_agent = Agent(name="Loan Agent", role="Loan Processor", goal="Help user apply for loan", tools=[loan_tool], ...)

# Define crew with the router as entry point
crew = Crew(
    agents=[router, balance_agent, loan_agent],
    entry_point=router
)

# Kick off the crew
crew.run("Hi, I'd like to apply for a loan")


# Chatting with model

In [112]:
def chat_with_openai(user_query):
    
    response = client.responses.create(
            model="gpt-4o-mini",
            input = user_query,
            
            instructions = combined_prompt,
            tools=function_descriptions
            )

    return response,response.output

In [113]:
def chat_balance():
    
    print(" Welcome! How can I help you Today? Type 'exit' to end the chat.\n")


    while True:
        
        user_query = input("You: ")
        
        messages = [{"role": "user", "content": user_query}]

        response, output = chat_with_openai(user_query)
        print(f"bot: {response.output_text}")
        
        tool_call = response.output[0]
        tool_call_list = list(response.output[0])
        tool_call_dict = dict(tool_call_list)
        arguments_str = tool_call_dict.get("arguments")
        call_id = tool_call_dict.get("call_id")

        if arguments_str:
            args = json.loads(arguments_str)
#             print(f"Parsed arguments: {args}")
        else:
            continue  
     
        customer_id = args['customer_id']
        
        function_results = get_balance(customer_id)
        
        # append model's function call message
        messages.append(tool_call_dict)
        
        
        # append result message
        messages.append({                             
            "type": "function_call_output",
            "call_id": call_id,
            "output": str(function_results)
        })
        

        response_2 = client.responses.create(
            model="gpt-4o-mini",
            input=messages,
            tools=function_descriptions,
        )
        print(response_2.output_text)
        
        


# Calling Chatbot

In [76]:
list(df_customers.id)[0:5]

['bd9c5311-c8f8-441d-b205-e68fb7aa7293',
 '4f8667b3-f381-44c4-8f3c-2a0718248284',
 'a217997d-21c3-4716-ac19-5d5c9950c260',
 'b2ad5dfb-e86e-460d-9f98-3351c040e02f',
 '9e5d54aa-168e-4347-8514-9ec8d1d5a470']

In [114]:
chat_balance()

 Welcome! How can I help you Today? Type 'exit' to end the chat.

You: hi what is my balance
bot: Could you please provide me with your customer ID?
You: bd9c5311-c8f8-441d-b205-e68fb7aa7293
bot: It seems you've provided a customer ID. How can I assist you today? Would you like to check your account balance or apply for a loan?
You: account balance please
bot: Could you please provide me with your customer ID?
You: bd9c5311-c8f8-441d-b205-e68fb7aa7293
bot: Could you please let me know how much you would like to borrow?
You: 2000
bot: Please provide your customer ID so I can check your account balance.
You: bd9c5311-c8f8-441d-b205-e68fb7aa7293
bot: Could you please let me know what you would like to do? You can either check your account balance or apply for a loan.


KeyboardInterrupt: Interrupted by user